In [2]:
import os,zipfile,shutil,requests

import torch
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim
import torch.jit as jit
import torch.nn.functional as F
import numpy as np
from torchvision import transforms
import lightning.pytorch as pl

In [6]:
import contextlib
import io
import random

In [7]:
# Initialize the probabilities for the hyperparameters
param_probabilities = { 
    'num_conv_layers': [0.3, 0.3, 0.2, 0.2],  # Probabilities for [2, 3, 4] layers
    'base_num_filters': [0.4, 0.4, 0.2],  # Probabilities for [16, 32, 64] filters
    'kernel_size': [0.5, 0.3, 0.2],  # Probabilities for [3, 5, 7] kernel sizes
    'dropout_rate': [0.6, 0.4],  # Probabilities for [0.3, 0.5] dropout rates
    'fc_units': [0.4, 0.4, 0.2],  # Probabilities for [128, 256, 512] units
    'activation_function': [0.25, 0.3, 0.2, 0.25],  # Probabilities for ['relu', 'leaky_relu', 'tanh']
    'use_batch_norm': [0.5, 0.5],  # Probabilities for [True, False]
    'use_residual_blocks': [0.5,0.5],
    'use_se_block': [0.5,0.5],
    'use_dropout_in_conv': [0.5,0.5],
    'multi_branch_block': [0.5,0.5],
    'use_dilated_conv': [0.5,0.5]
}

In [ ]:
import numpy as np
import torch.nn as nn
import torch.nn.functional as F

import json

class SEBlock(nn.Module):
    def __init__(self, in_channels, reduction=16):
        super(SEBlock, self).__init__()
        self.fc1 = nn.Linear(in_channels, in_channels // reduction)
        self.fc2 = nn.Linear(in_channels // reduction, in_channels)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        batch, channels, _, _ = x.size()
        y = torch.mean(x, dim=(2, 3))  # Global average pooling
        y = self.fc1(y)
        y = self.relu(y)
        y = self.fc2(y)
        y = self.sigmoid(y).view(batch, channels, 1, 1)
        return x * y  # Scale the input
        



# Function to dynamically create a CNN model based on parameters
def generate_dynamic_cnn(num_classes, params):
    """
    Generate a CNN model dynamically based on the provided parameters.
    
    Args:
        num_classes (int): Number of classes for classification.
        params (dict): Dictionary of hyperparameters to control model creation.

    Returns:
        nn.Module: A dynamically created CNN model.
    """
    class DynamicCNN(nn.Module):
        def __init__(self):
            super(DynamicCNN, self).__init__()
            layers = []
            in_channels = 3  # Starting with an input of 3 channels (RGB)

            # Create convolutional layers based on the specified number of layers
            for i in range(params['num_conv_layers']):
                out_channels = params['base_num_filters'] * (2 ** i)

                if params['use_dilated_conv']:
                    layers.append(nn.Conv2d(in_channels, out_channels, kernel_size=params['kernel_size'], padding=2, dilation=2))
                else:
                    layers.append(nn.Conv2d(in_channels, out_channels, kernel_size=params['kernel_size'], padding=1))

                if params['use_batch_norm']:
                    layers.append(nn.BatchNorm2d(out_channels))

                layers.append(get_activation_function(params['activation_function']))
                
                layers.append(nn.MaxPool2d(kernel_size=params['pool_size'], stride=2))

                
            

                # Add SEBlock after convolutional layers
                if params['use_se_block']:
                    layers.append(SEBlock(out_channels))
                    
                if params['use_dropout_in_conv']:
                    layers.append(nn.Dropout2d(params['dropout_rate']))

                    # Add the MultiBranchBlock in your model
                
                in_channels = out_channels

            self.conv_layers = nn.Sequential(*layers)

            # Calculate the size of the flattened input for the fully connected layer
            flattened_size = out_channels * (32 // (2 ** params['num_conv_layers'])) ** 2

            # Define fully connected layers
            self.fc1 = nn.Linear(flattened_size, params['fc_units'])
            self.dropout = nn.Dropout(params['dropout_rate'])
            self.fc2 = nn.Linear(params['fc_units'], num_classes)

        def forward(self, x):
            x = self.conv_layers(x)
            x = x.view(x.size(0), -1)  # Flatten the tensor for the dense layers
            x = get_activation_function(params['activation_function'])(self.fc1(x))
            x = self.dropout(x)
            x = self.fc2(x)
            return x

    return DynamicCNN()

def get_activation_function(name):
    """Returns the activation function based on the given name."""
    if name == 'relu':
        return nn.ReLU()
    elif name == 'leaky_relu':
        return nn.LeakyReLU()
    elif name == 'tanh':
        return nn.Tanh()
    elif name == 'sigmoid':
        return nn.Sigmoid()
    else:
        raise ValueError(f"Unsupported activation function: {name}")



# Function to sample a parameter value based on its probabilities
def sample_parameter(options, probabilities):
    return np.random.choice(options, p=probabilities)

# Function to update probabilities based on performance
def update_probabilities(param_probabilities, performance_score, last_params):
    """
    Update the probabilities of the parameters based on the model's performance.
    Args:
        param_probabilities (dict): Current parameter probabilities.
        performance_score (float): Accuracy or validation loss to determine performance.
        last_params (dict): Parameters of the last trained model.

    Returns:
        dict: Updated parameter probabilities.
    """
    for key in last_params:
        if performance_score > 0.75:  # Threshold to favor good parameters
            param_probabilities[key] = [p + 0.1 if option == last_params[key] else p - 0.05 for p, option in zip(param_probabilities[key], [2, 3, 4])]
        else:  # Lower the probability of less effective parameters
            param_probabilities[key] = [p - 0.05 if option == last_params[key] else p + 0.05 for p, option in zip(param_probabilities[key], [2, 3, 4])]

    # Ensure that the probabilities remain in a valid range (0 to 1) and sum to 1
    for key in param_probabilities:
        total = sum(param_probabilities[key])
        param_probabilities[key] = [p / total for p in param_probabilities[key]]

    return param_probabilities

try:
    all_models = load_dict_from_file()
except:
    all_models = {}

# Loop to create, train, and evaluate 10 different models
try:
    for i in range(100):
        # Sample parameters for the next model based on the current probabilities
        params = {
            'num_conv_layers': sample_parameter([2, 3, 4, 5], param_probabilities['num_conv_layers']),
            'base_num_filters': sample_parameter([16, 32, 64], param_probabilities['base_num_filters']),
            'kernel_size': 3,
            'dropout_rate': round(random.random()*0.5,2),
            'fc_units': sample_parameter([128, 256, 512], param_probabilities['fc_units']),
            'activation_function': sample_parameter(['relu', 'leaky_relu', 'tanh','sigmoid'], param_probabilities['activation_function']),
            'use_batch_norm': sample_parameter([True, False], param_probabilities['use_batch_norm']),
            'pool_size': 2, 
            'use_se_block': sample_parameter([True, False], param_probabilities['use_se_block']),
            'use_dropout_in_conv': sample_parameter([True, False], param_probabilities['use_dropout_in_conv']),
            'use_dilated_conv': sample_parameter([True, False], param_probabilities['use_dilated_conv'])
        }

        if params['num_conv_layers'] > 4:
            params['base_num_filters'] = 32

        print(f"\nCreating and training model {i + 1} with parameters: {params}")
        while True:
            model_ID = hash(random.random())
            if model_ID not in all_models.keys():
                break

        # Dynamically generate a CNN model
        model = generate_dynamic_cnn(num_classes=train_dataloader.dataset.num_classes, params=params)

       ################################


       ################################


       
        all_models[model_ID] ={
            'params': params,
            'accuracy': response_dict['eval_accuracy'],
            'misc':response_dict
        }
except KeyboardInterrupt:
    print("KeyboardInterrupt caught! Saving all_models to file...")
    
save_dict_to_file(all_models)  # Save the dictionary to the file on interruption
print("Saved successfully!")

INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]
INFO: 
  | Name  | Type       | Params | Mode 
---------------------------------------------
0 | model | DynamicCNN | 4.3 M  | train
---------------------------------------------
4.3 M     Trainable params
0         Non-trainable params
4.3 M     Total params
17.138    Total estimated model params size (MB)



Creating and training model 1 with parameters: {'num_conv_layers': 2, 'base_num_filters': 64, 'kernel_size': 3, 'dropout_rate': 0.26, 'fc_units': 512, 'activation_function': 'leaky_relu', 'use_batch_norm': True, 'pool_size': 2, 'use_se_block': True, 'use_dropout_in_conv': False, 'use_dilated_conv': False}


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 672 / 672 (100.0%)
Greatest absolute difference: 2.859885811805725 at index (4, 20) (up to 1e-05 allowed)
Greatest relative difference: 87.42314795010832 at index (26, 0) (up to 1e-05 allowed)
  _check_trace(
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]
INFO: 
  | Name  | Type       | Params | Mode 
---------------------------------------------
0 | model | DynamicCNN | 626 K  | train
---------------------------------------------
626 K     Trainable params
0         Non-trainable params
626 K     Total params
2.506     Total estimated model params size (MB)


{'eval_accuracy': '70.30%', 'model_size (MB)': '4.15', 'percentile (Here is where you stand against the community)': '19th percentile', 'submission_id': 'E=2024-10-22T05:50:08.739736_68244776497'}

Creating and training model 28 with parameters: {'num_conv_layers': 3, 'base_num_filters': 32, 'kernel_size': 3, 'dropout_rate': 0.47, 'fc_units': 256, 'activation_function': 'tanh', 'use_batch_norm': True, 'pool_size': 2, 'use_se_block': True, 'use_dropout_in_conv': False, 'use_dilated_conv': True}


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]